Connected to Python 3.11.7

In [1]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length, hidden_channels, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(CategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, kernel_size in zip(hidden_channels, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.Conv1d(prev_channels, latent_dim, kernel_size=1))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 32, 64]
    kernel_sizes = [5, 5, 5]
    latent_dim = 64

    autoencoder = ConvolutionalCategoricalAutoencoder(
        # input_channels, input_length, hidden_channels, kernel_sizes, latent_dim,
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [2]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length, hidden_channels, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(ConvolutionalCategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, kernel_size in zip(hidden_channels, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.Conv1d(prev_channels, latent_dim, kernel_size=1))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 32, 64]
    kernel_sizes = [5, 5, 5]
    latent_dim = 64

    autoencoder = ConvolutionalCategoricalAutoencoder(
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

ConvolutionalCategoricalAutoencoder(
  (activation): LeakyReLU(negative_slope=0.01)
  (latent_activation): Identity()
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(2,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(16, 32, kernel_size=(5,), stride=(2,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(32, 64, kernel_size=(5,), stride=(2,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    (7): Identity()
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(64, 64, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose1d(32, 16, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): C

In [3]:
p = self.encode(x)

NameError: name 'self' is not defined

In [4]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length, hidden_channels, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(ConvolutionalCategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, kernel_size in zip(hidden_channels, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.Conv1d(prev_channels, latent_dim, kernel_size=1))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        print(p.size())
        return p
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 32, 64]
    kernel_sizes = [5, 5, 5]
    latent_dim = 64

    autoencoder = ConvolutionalCategoricalAutoencoder(
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

ConvolutionalCategoricalAutoencoder(
  (activation): LeakyReLU(negative_slope=0.01)
  (latent_activation): Identity()
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(2,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(16, 32, kernel_size=(5,), stride=(2,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(32, 64, kernel_size=(5,), stride=(2,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    (7): Identity()
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(64, 64, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose1d(32, 16, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): C

In [5]:
autoencoder(torch.randn(64, 1, 200))

torch.Size([64, 64, 24])


tensor([[[-0.1073, -0.1115, -0.1075,  ..., -0.1236, -0.1223, -0.1728],
         [-0.0858, -0.0914, -0.0766,  ..., -0.0272, -0.0955, -0.0508],
         [-0.0437, -0.0297, -0.0391,  ..., -0.0260, -0.0033, -0.0131],
         ...,
         [-0.0491, -0.0589, -0.0766,  ..., -0.0683, -0.0319, -0.0650],
         [-0.0364, -0.0838, -0.0981,  ..., -0.0477, -0.0723, -0.1099],
         [-0.0244, -0.0408, -0.0766,  ..., -0.0615, -0.0137, -0.0135]],

        [[-0.1078, -0.1388, -0.1500,  ..., -0.0940, -0.0858, -0.1200],
         [-0.0717, -0.1100, -0.0860,  ..., -0.0814, -0.0907, -0.0799],
         [-0.0360, -0.0429, -0.0480,  ..., -0.0768, -0.0655, -0.0292],
         ...,
         [-0.0861, -0.0401, -0.0617,  ..., -0.0444, -0.0716, -0.0124],
         [-0.0974, -0.0964, -0.1060,  ..., -0.0569, -0.0995, -0.0908],
         [-0.0645, -0.0706, -0.0610,  ..., -0.1090, -0.0796, -0.0193]],

        [[-0.1082, -0.0666, -0.0803,  ..., -0.0886, -0.0841, -0.1090],
         [-0.1283, -0.1053, -0.0668,  ..., -0

In [6]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length, hidden_channels, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(ConvolutionalCategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, kernel_size in zip(hidden_channels, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.Conv1d(prev_channels, latent_dim, kernel_size=1))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        # p = self.encode(x)
        for l in self.encoder:
            x = l(x)
            print(x.size())
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 32, 64]
    kernel_sizes = [5, 5, 5]
    latent_dim = 64

    autoencoder = ConvolutionalCategoricalAutoencoder(
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

ConvolutionalCategoricalAutoencoder(
  (activation): LeakyReLU(negative_slope=0.01)
  (latent_activation): Identity()
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(2,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(16, 32, kernel_size=(5,), stride=(2,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(32, 64, kernel_size=(5,), stride=(2,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    (7): Identity()
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(64, 64, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose1d(32, 16, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): C

In [7]:
autoencoder(torch.randn(64, 1, 200))

torch.Size([64, 16, 99])
torch.Size([64, 16, 99])
torch.Size([64, 32, 49])
torch.Size([64, 32, 49])
torch.Size([64, 64, 24])
torch.Size([64, 64, 24])
torch.Size([64, 64, 24])
torch.Size([64, 64, 24])


NameError: name 'p' is not defined

In [8]:
autoencoder(torch.randn(64, 1, 200))

torch.Size([64, 16, 99])
torch.Size([64, 16, 99])
torch.Size([64, 32, 49])
torch.Size([64, 32, 49])
torch.Size([64, 64, 24])
torch.Size([64, 64, 24])
torch.Size([64, 64, 24])
torch.Size([64, 64, 24])


NameError: name 'p' is not defined

In [9]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length, hidden_channels, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(ConvolutionalCategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, kernel_size in zip(hidden_channels, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.Conv1d(prev_channels, latent_dim, kernel_size=1))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        # p = self.encode(x)
        for l in self.encoder:
            x = l(x)
            print(x.size())
        return
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 32, 64]
    kernel_sizes = [5, 5, 5]
    latent_dim = 64

    autoencoder = ConvolutionalCategoricalAutoencoder(
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

ConvolutionalCategoricalAutoencoder(
  (activation): LeakyReLU(negative_slope=0.01)
  (latent_activation): Identity()
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(2,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(16, 32, kernel_size=(5,), stride=(2,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(32, 64, kernel_size=(5,), stride=(2,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    (7): Identity()
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(64, 64, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose1d(32, 16, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): C

In [10]:
autoencoder(torch.randn(64, 1, 200))

torch.Size([64, 16, 99])
torch.Size([64, 16, 99])
torch.Size([64, 32, 49])
torch.Size([64, 32, 49])
torch.Size([64, 64, 24])
torch.Size([64, 64, 24])
torch.Size([64, 64, 24])
torch.Size([64, 64, 24])


In [11]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length,
                 hidden_channels, strides, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(ConvolutionalCategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, stride, kernel_size in zip(hidden_channels, strides, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=stride, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.Conv1d(prev_channels, latent_dim, kernel_size=1))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        # p = self.encode(x)
        for l in self.encoder:
            x = l(x)
            print(x.size())
        return
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 16, 32, 32, 64, 64]
    strides = [1, 2, 1, 2, 1, 2]
    kernel_sizes = [5, 5, 5, 5, 5, 5]
    latent_dim = 64

    autoencoder = ConvolutionalCategoricalAutoencoder(
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        strides=strides,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

ConvolutionalCategoricalAutoencoder(
  (activation): LeakyReLU(negative_slope=0.01)
  (latent_activation): Identity()
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(1,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(16, 16, kernel_size=(5,), stride=(2,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv1d(32, 32, kernel_size=(5,), stride=(2,), padding=(1,))
    (7): LeakyReLU(negative_slope=0.01)
    (8): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(1,))
    (9): LeakyReLU(negative_slope=0.01)
    (10): Conv1d(64, 64, kernel_size=(5,), stride=(2,), padding=(1,))
    (11): LeakyReLU(negative_slope=0.01)
    (12): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    (13): Identity()
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(64, 64, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,

In [12]:
autoencoder(torch.randn(64, 1, 200))

torch.Size([64, 16, 198])
torch.Size([64, 16, 198])
torch.Size([64, 16, 98])
torch.Size([64, 16, 98])
torch.Size([64, 32, 96])
torch.Size([64, 32, 96])
torch.Size([64, 32, 47])
torch.Size([64, 32, 47])
torch.Size([64, 64, 45])
torch.Size([64, 64, 45])
torch.Size([64, 64, 22])
torch.Size([64, 64, 22])
torch.Size([64, 64, 22])
torch.Size([64, 64, 22])


In [13]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length,
                 hidden_channels, strides, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(ConvolutionalCategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, stride, kernel_size in zip(hidden_channels, strides, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=stride, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.Conv1d(prev_channels, latent_dim, kernel_size=1))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        # p = self.encode(x)
        for l in self.encoder:
            x = l(x)
            print(x.size())
        return
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 32, 64]
    strides = [3, 3, 3]
    kernel_sizes = [5, 5, 5]
    latent_dim = 64

    autoencoder = ConvolutionalCategoricalAutoencoder(
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        strides=strides,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

ConvolutionalCategoricalAutoencoder(
  (activation): LeakyReLU(negative_slope=0.01)
  (latent_activation): Identity()
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(3,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(16, 32, kernel_size=(5,), stride=(3,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(32, 64, kernel_size=(5,), stride=(3,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    (7): Identity()
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(64, 64, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose1d(32, 16, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): C

In [14]:
autoencoder(torch.randn(64, 1, 200))

torch.Size([64, 16, 66])
torch.Size([64, 16, 66])
torch.Size([64, 32, 22])
torch.Size([64, 32, 22])
torch.Size([64, 64, 7])
torch.Size([64, 64, 7])
torch.Size([64, 64, 7])
torch.Size([64, 64, 7])


In [15]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length,
                 hidden_channels, strides, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(ConvolutionalCategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, stride, kernel_size in zip(hidden_channels, strides, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=stride, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.AdaptiveMaxPool1d(1))
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        # p = self.encode(x)
        for l in self.encoder:
            x = l(x)
            print(x.size())
        return
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 32, 64]
    strides = [3, 3, 3]
    kernel_sizes = [5, 5, 5]
    latent_dim = 64

    autoencoder = ConvolutionalCategoricalAutoencoder(
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        strides=strides,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

ConvolutionalCategoricalAutoencoder(
  (activation): LeakyReLU(negative_slope=0.01)
  (latent_activation): Identity()
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(3,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(16, 32, kernel_size=(5,), stride=(3,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(32, 64, kernel_size=(5,), stride=(3,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): AdaptiveMaxPool1d(output_size=1)
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(64, 64, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose1d(32, 16, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): ConvTranspose1d(16, 1, kernel_size

In [16]:
autoencoder(torch.randn(64, 1, 200))

torch.Size([64, 16, 66])
torch.Size([64, 16, 66])
torch.Size([64, 32, 22])
torch.Size([64, 32, 22])
torch.Size([64, 64, 7])
torch.Size([64, 64, 7])
torch.Size([64, 64, 1])


In [17]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length,
                 hidden_channels, strides, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(ConvolutionalCategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, stride, kernel_size in zip(hidden_channels, strides, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=stride, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.AdaptiveMaxPool1d(1))
        encoder_layers.append(torch.nn.Linear(prev_channels, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        # p = self.encode(x)
        for l in self.encoder:
            x = l(x)
            print(x.size())
        return
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 32, 64]
    strides = [3, 3, 3]
    kernel_sizes = [5, 5, 5]
    latent_dim = 64

    autoencoder = ConvolutionalCategoricalAutoencoder(
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        strides=strides,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

ConvolutionalCategoricalAutoencoder(
  (activation): LeakyReLU(negative_slope=0.01)
  (latent_activation): Identity()
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(3,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(16, 32, kernel_size=(5,), stride=(3,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(32, 64, kernel_size=(5,), stride=(3,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): AdaptiveMaxPool1d(output_size=1)
    (7): Linear(in_features=64, out_features=64, bias=True)
    (8): Identity()
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(64, 64, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose1d(32, 16, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
   

In [18]:
autoencoder(torch.randn(64, 1, 200))

torch.Size([64, 16, 66])
torch.Size([64, 16, 66])
torch.Size([64, 32, 22])
torch.Size([64, 32, 22])
torch.Size([64, 64, 7])
torch.Size([64, 64, 7])
torch.Size([64, 64, 1])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (4096x1 and 64x64)

In [19]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length,
                 hidden_channels, strides, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(ConvolutionalCategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, stride, kernel_size in zip(hidden_channels, strides, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=stride, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.AdaptiveMaxPool1d(1))
        encoder_layers.append(torch.nn.Flatten())
        encoder_layers.append(torch.nn.Linear(prev_channels, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        # p = self.encode(x)
        for l in self.encoder:
            x = l(x)
            print(x.size())
        return
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 32, 64]
    strides = [3, 3, 3]
    kernel_sizes = [5, 5, 5]
    latent_dim = 64

    autoencoder = ConvolutionalCategoricalAutoencoder(
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        strides=strides,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

ConvolutionalCategoricalAutoencoder(
  (activation): LeakyReLU(negative_slope=0.01)
  (latent_activation): Identity()
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(3,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(16, 32, kernel_size=(5,), stride=(3,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(32, 64, kernel_size=(5,), stride=(3,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): AdaptiveMaxPool1d(output_size=1)
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): Linear(in_features=64, out_features=64, bias=True)
    (9): Identity()
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(64, 64, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose1d(32, 16, kernel_size=(5,), stride=(2

In [20]:
autoencoder(torch.randn(64, 1, 200))

torch.Size([64, 16, 66])
torch.Size([64, 16, 66])
torch.Size([64, 32, 22])
torch.Size([64, 32, 22])
torch.Size([64, 64, 7])
torch.Size([64, 64, 7])
torch.Size([64, 64, 1])
torch.Size([64, 64])
torch.Size([64, 64])
torch.Size([64, 64])


In [21]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length,
                 hidden_channels, strides, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(ConvolutionalCategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, stride, kernel_size in zip(hidden_channels, strides, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=stride, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.AdaptiveMaxPool1d(1))
        encoder_layers.append(torch.nn.Flatten())
        encoder_layers.append(torch.nn.Linear(prev_channels, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        # p = self.encode(x)
        for l in self.encoder:
            x = l(x)
            print(x.size())
        return
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 32, 64]
    strides = [3, 3, 3]
    kernel_sizes = [5, 5, 5]
    latent_dim = 16

    autoencoder = ConvolutionalCategoricalAutoencoder(
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        strides=strides,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

ConvolutionalCategoricalAutoencoder(
  (activation): LeakyReLU(negative_slope=0.01)
  (latent_activation): Identity()
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(3,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(16, 32, kernel_size=(5,), stride=(3,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(32, 64, kernel_size=(5,), stride=(3,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): AdaptiveMaxPool1d(output_size=1)
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): Linear(in_features=64, out_features=16, bias=True)
    (9): Identity()
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(16, 64, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose1d(32, 16, kernel_size=(5,), stride=(2

In [22]:
autoencoder(torch.randn(64, 1, 200))

torch.Size([64, 16, 66])
torch.Size([64, 16, 66])
torch.Size([64, 32, 22])
torch.Size([64, 32, 22])
torch.Size([64, 64, 7])
torch.Size([64, 64, 7])
torch.Size([64, 64, 1])
torch.Size([64, 64])
torch.Size([64, 16])
torch.Size([64, 16])


In [23]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length,
                 hidden_channels, strides, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(ConvolutionalCategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, stride, kernel_size in zip(hidden_channels, strides, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=stride, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.AdaptiveMaxPool1d(1))
        encoder_layers.append(torch.nn.Flatten())
        encoder_layers.append(torch.nn.Linear(prev_channels, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        print(p.size())
        z = self.reparameterize(p, temperature=self.temperature)
        return
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 32, 64]
    strides = [3, 3, 3]
    kernel_sizes = [5, 5, 5]
    latent_dim = 16

    autoencoder = ConvolutionalCategoricalAutoencoder(
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        strides=strides,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

ConvolutionalCategoricalAutoencoder(
  (activation): LeakyReLU(negative_slope=0.01)
  (latent_activation): Identity()
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(3,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(16, 32, kernel_size=(5,), stride=(3,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(32, 64, kernel_size=(5,), stride=(3,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): AdaptiveMaxPool1d(output_size=1)
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): Linear(in_features=64, out_features=16, bias=True)
    (9): Identity()
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(16, 64, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose1d(32, 16, kernel_size=(5,), stride=(2

In [24]:
autoencoder(torch.randn(64, 1, 200))

torch.Size([64, 16])


In [25]:
autoencoder(torch.randn(64, 1, 200))[0]

torch.Size([64, 16])


TypeError: 'NoneType' object is not subscriptable

In [26]:
import torch

class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(Autoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)

        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def vae_loss(recon_x, x, mu, logvar, kl_weight=1E-3):
    BCE = torch.nn.functional.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD


class VariationalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, activation='lrelu', latent_activation='lrelu', negative_slope=0.01):
        super(VariationalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        self.encoder = torch.nn.Sequential(*encoder_layers)
        self.fc_mu = torch.nn.Linear(prev_dim, latent_dim)
        self.fc_logvar = torch.nn.Linear(prev_dim, latent_dim)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def gumbel_elbo_loss(X_pred, X_true, p, kl_weight=1E-3):
    rec_loss = torch.nn.functional.mse_loss(X_pred, X_true)
    logits = torch.nn.functional.log_softmax(p, dim=-1)
    kl = torch.nn.functional.kl_div(logits, torch.ones_like(logits) / p.size()[-1])
    return rec_loss + kl_weight * kl

class CategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=.5):
        super(CategoricalAutoencoder, self).__init__()
        
        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            encoder_layers.append(self.activation)
            prev_dim = hidden_dim
        encoder_layers.append(torch.nn.Linear(prev_dim, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = latent_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(torch.nn.Linear(prev_dim, hidden_dim))
            decoder_layers.append(self.activation)
            prev_dim = hidden_dim
        decoder_layers.append(torch.nn.Linear(prev_dim, input_dim))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


class ConvolutionalCategoricalAutoencoder(torch.nn.Module):
    def __init__(self, input_channels, input_length,
                 hidden_channels, strides, kernel_sizes, latent_dim,
                 activation='lrelu', latent_activation='lrelu',
                 negative_slope=0.01, temperature=0.5):
        super(ConvolutionalCategoricalAutoencoder, self).__init__()

        self.activation = self._get_activation_function(activation, negative_slope)
        self.latent_activation = self._get_activation_function(latent_activation, negative_slope)
        self.latent_dim = latent_dim
        self.temperature = temperature

        # Encoder
        encoder_layers = []
        prev_channels = input_channels
        for hidden_channel, stride, kernel_size in zip(hidden_channels, strides, kernel_sizes):
            encoder_layers.append(torch.nn.Conv1d(prev_channels, hidden_channel, kernel_size, stride=stride, padding=1))
            encoder_layers.append(self.activation)
            prev_channels = hidden_channel
        encoder_layers.append(torch.nn.AdaptiveMaxPool1d(1))
        encoder_layers.append(torch.nn.Flatten())
        encoder_layers.append(torch.nn.Linear(prev_channels, latent_dim))
        encoder_layers.append(self.latent_activation)
        self.encoder = torch.nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_channels = latent_dim
        reversed_hidden_channels = list(reversed(hidden_channels))
        reversed_kernel_sizes = list(reversed(kernel_sizes))
        for hidden_channel, kernel_size in zip(reversed_hidden_channels, reversed_kernel_sizes):
            decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, hidden_channel, kernel_size, stride=2, padding=1, output_padding=1))
            decoder_layers.append(self.activation)
            prev_channels = hidden_channel
        decoder_layers.append(torch.nn.ConvTranspose1d(prev_channels, input_channels, kernel_size=1))
        self.decoder = torch.nn.Sequential(*decoder_layers)


    def _get_activation_function(self, activation, negative_slope):
        if activation == 'lrelu':
            return torch.nn.LeakyReLU(negative_slope=negative_slope)
        elif activation == 'relu':
            return torch.nn.ReLU()
        elif activation == 'sigmoid':
            return torch.nn.Sigmoid()
        elif activation == 'tanh':
            return torch.nn.Tanh()
        elif activation == 'linear':
            return torch.nn.Identity()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

    def encode(self, x):
        p = self.encoder(x)
        return p

    def reparameterize(self, p, temperature=0.5, epsilon=1E-7):
        g = torch.rand_like(p)
        g = -torch.log(-torch.log(g + epsilon) + epsilon)

        z = torch.nn.functional.softmax((p + g) / temperature, dim=-1)
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        p = self.encode(x)
        print(p.size())
        z = self.reparameterize(p, temperature=self.temperature)
        return self.decode(z), p


if __name__ == '__main__':
    input_dim = 200
    hidden_channels = [16, 32, 64]
    strides = [3, 3, 3]
    kernel_sizes = [5, 5, 5]
    latent_dim = 16

    autoencoder = ConvolutionalCategoricalAutoencoder(
        input_channels=1,
        input_length=input_dim,
        hidden_channels=hidden_channels,
        strides=strides,
        kernel_sizes=kernel_sizes,
        latent_dim=latent_dim,
        activation='lrelu',
        latent_activation='linear',
        negative_slope=0.01
    )

    print(autoencoder)

ConvolutionalCategoricalAutoencoder(
  (activation): LeakyReLU(negative_slope=0.01)
  (latent_activation): Identity()
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(3,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(16, 32, kernel_size=(5,), stride=(3,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(32, 64, kernel_size=(5,), stride=(3,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): AdaptiveMaxPool1d(output_size=1)
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): Linear(in_features=64, out_features=16, bias=True)
    (9): Identity()
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(16, 64, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(2,), padding=(1,), output_padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose1d(32, 16, kernel_size=(5,), stride=(2

In [27]:
autoencoder(torch.randn(64, 1, 200))[0]

torch.Size([64, 16])


RuntimeError: Given transposed=1, weight of size [16, 64, 5], expected input[1, 64, 16] to have 16 channels, but got 64 channels instead